In [1]:
import os
import time
import json
import pickle
import timeit
import random
import param
import shutil
import collections
import numpy as np
import tensorflow as tf

import stctrain
import datahelper
import stctokenizer
import nuggetdetection as ND
import dialogquality as DQ
import dialogquality_ndfeature as DQNDF
import stcevaluation as STCE

from scipy import stats
from collections import Counter
from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors

import logging
logging.basicConfig(level=logging.INFO)

Using TensorFlow backend.


In [2]:
doclen = param.doclen
embsize = param.embsize
max_sent = param.max_sent
NDclasses = param.NDclasses
DQclasses = param.DQclasses
sentembsize = param.sentembsize

REMOVE_STOPWORDS = False
TO_LOWER = True
TOKEN_TYPE = 'nltk'

datahelper = datahelper.DataHelper(embedding_path="../embedding/STCWiki/STCWiki_mincount0.model.bin")
stctokenizer = stctokenizer.STCTokenizer()

INFO:gensim.utils:loading Word2VecKeyedVectors object from ../embedding/STCWiki/STCWiki_mincount0.model.bin
INFO:gensim.utils:loading vectors from ../embedding/STCWiki/STCWiki_mincount0.model.bin.vectors.npy with mmap=None
INFO:gensim.utils:setting ignored attribute vectors_norm to None
INFO:gensim.utils:loaded ../embedding/STCWiki/STCWiki_mincount0.model.bin


## Word Embedding

In [3]:
# corpus = datahelper.prepare_word_embedding_corpus(
#     '../data/text8', 
#     TOKEN_TYPE, 
#     REMOVE_STOPWORDS, 
#     TO_LOWER,
# )

# wordemb_model = Word2Vec(corpus, size=300, min_count=0, workers=4, iter=25)
# word_vectors = wordemb_model.wv
# datahelper.test_w2v_model(word_vectors, word='problem', topn=10)

# 儲存訓練好的word2vec模型，因為已經訓練並儲存好了所以註解掉
# word_vectors.save("../embedding/STCWiki/STCWiki_mincount0.model.bin")
# datahelper.set_word_vectors(word_vectors)

## Read Corpus & Prepare data for model

In [4]:
trainX, trainND, trainDQ, train_turns, train_masks = datahelper.get_model_train_data(
    'train',
    TOKEN_TYPE, 
    REMOVE_STOPWORDS, 
    TO_LOWER,
)

devX, devND, devDQ, dev_turns, dev_masks = datahelper.get_model_train_data(
    'dev',
    TOKEN_TYPE, 
    REMOVE_STOPWORDS, 
    TO_LOWER,
)

testX, test_turns, test_masks = datahelper.get_model_test_data(token_type='nltk', remove_stopwords=False, to_lower=True)

testIDs = datahelper.testIDs
trainDQA = [item['A'] for item in trainDQ]
trainDQS = [item['S'] for item in trainDQ]
trainDQE = [item['E'] for item in trainDQ]
devDQA = [item['A'] for item in devDQ]
devDQS = [item['S'] for item in devDQ]
devDQE = [item['E'] for item in devDQ]

INFO:corpus word2vec:Training data unknown words count: 0
INFO:corpus word2vec:Training data unknown words count: 0
INFO:corpus word2vec:Testing data unknown words count: 0


In [5]:
dataND = [trainX, trainND, train_turns, train_masks, devX, devND, dev_turns, dev_masks, testX, test_turns, test_masks]
dataDQA = [trainX, trainDQA, train_turns, devX, devDQA, dev_turns, testX, test_turns]
dataDQS = [trainX, trainDQS, train_turns, devX, devDQS, dev_turns, testX, test_turns]
dataDQE = [trainX, trainDQE, train_turns, devX, devDQE, dev_turns, testX, test_turns]

In [6]:
fixed_paramsND  = {'epoch':50, 'early_stopping':50, 'batch_size':30, 'lr':0.0005, 'kp':1, 'hiddens':1024, 'Fsize':[2,2]} 
fixed_paramsDQA = {'scoretype':'DQA', 'epoch':50, 'early_stopping':3, 'batch_size':40, 'lr':0.001, 'kp':1, 'hiddens':1024, 'Fsize':[2,2]} 
fixed_paramsDQS = {'scoretype':'DQS', 'epoch':50, 'early_stopping':3, 'batch_size':40, 'lr':0.001, 'kp':1, 'hiddens':1024, 'Fsize':[2,2]} 
fixed_paramsDQE = {'scoretype':'DQE', 'epoch':50, 'early_stopping':3, 'batch_size':40, 'lr':0.001, 'kp':1, 'hiddens':1024, 'Fsize':[2,2]} 

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(title, train, valid):
    x = [i for i in range(1, len(train)+1)]
    plt.rcParams.update({'font.size': 22})
    plt.plot(x, train, marker='o', linestyle='-', color='b')
    plt.plot(x, valid, marker='o', linestyle='-', color='r')
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
    plt.show()

## Test ND

In [ ]:
bestNDs = []
train_losses_list = []
dev_losses_list = []
e = False
for fn in [[256], [256,512], [256,512,1024]]:
    testND, train_losses, dev_losses = stctrain.start_trainND(
        *dataND, 
        **fixed_paramsND, 
        Fnum=fn, gating=False, bn=True, method=ND.CNNRNN, 
        evaluate=e,
    )

    bestNDs.append(testND)
    train_losses_list.append(train_losses)
    dev_losses_list.append(dev_losses)

In [ ]:
show_train_history('1-stack CNN + RNN', train_losses_list[0], dev_losses_list[0])
show_train_history('2-stack CNN + RNN', train_losses_list[1], dev_losses_list[1])
show_train_history('3-stack CNN + RNN', train_losses_list[2], dev_losses_list[2])

In [ ]:
# pickle.dump(bestNDs, open('memoryND_Labels.p', 'wb'))

## Test DQ

In [ ]:
bestDQAs = []
bestDQSs = []
bestDQEs = []

train_losses_listA = []
dev_losses_listA = []
e = False
for fn in [[512, 1024], [256, 512, 1024]]:
    for m in [DQ.CNNCNN, DQ.CNNRNN]:
        bestDQA, train_lossesA, dev_lossesA = stctrain.start_trainDQ(
            *dataDQA, 
            **fixed_paramsDQA, 
            Fnum=fn, gating=True, bn=True, method=m, 
            evaluate=e,
        )

#         bestDQS = stctrain.start_trainDQ(
#             *dataDQS, 
#             **fixed_paramsDQS, 
#             Fnum=fn, gating=True, bn=bn, method=DQ.CNNRNN, 
#             evaluate=e
#         )

#         bestDQE = stctrain.start_trainDQ(
#             *dataDQE, 
#             **fixed_paramsDQE, 
#             Fnum=fn, gating=True, bn=bn, method=DQ.CNNRNN, 
#             evaluate=e,
#         )
        train_losses_listA.append(train_lossesA)
        dev_losses_listA.append(dev_lossesA)
        bestDQAs.append(bestDQA)
    #         bestDQSs.append(bestDQS)
    #         bestDQEs.append(bestDQE)
        print('-')

INFO:Training:CNNCNN|14|True|True|2_2|1024|512_1024|0.08473|0.13266


-


In [ ]:
show_train_history('2-stack CNN + CNN', train_losses_listA[0], dev_losses_listA[0])
show_train_history('2-stack CNN + RNN', train_losses_listA[1], dev_losses_listA[1])

In [ ]:
# pickle.dump(bestDQAs, open('memoryDQAs.p', 'wb'))
# pickle.dump(bestDQSs, open('memoryDQSs.p', 'wb'))
# pickle.dump(bestDQEs, open('memoryDQEs.p', 'wb'))

## Test NDF

In [ ]:
memoryNDs = pickle.load(open('memoryNDs.p', 'rb'))

In [ ]:
testNDmasked = [np.multiply(nd, mask) for nd, mask in zip(memoryNDs[1], test_masks)]

In [ ]:
dataDQA_NDF = [trainX, trainDQA, train_turns, trainND, devX, devDQA, dev_turns, devND, testX, test_turns, testNDmasked]
dataDQS_NDF = [trainX, trainDQS, train_turns, trainND, devX, devDQS, dev_turns, devND, testX, test_turns, testNDmasked]
dataDQE_NDF = [trainX, trainDQE, train_turns, trainND, devX, devDQE, dev_turns, devND, testX, test_turns, testNDmasked]

In [ ]:
bestDQANDFs = []
bestDQSNDFs = []
bestDQENDFs = []
e = True
for fn in [[128, 256, 512, 1024], [128, 256, 512, 1024, 1024], [128, 256, 512, 1024, 1024, 1024]]:
#     bestDQANDF = stctrain.start_trainDQ_NDF(
#         *dataDQA_NDF, 
#         **fixed_paramsDQA, 
#         Fnum=fn, gating=True, bn=bn, method=DQNDF.CNNRNN, 
#         evaluate=e,
#     )

    bestDQSNDF = stctrain.start_trainDQ_NDF(
        *dataDQS_NDF, 
        **fixed_paramsDQS, 
        Fnum=fn, gating=True, bn=True, method=DQNDF.CNNRNN, 
        evaluate=e
    )

    bestDQENDF = stctrain.start_trainDQ_NDF(
        *dataDQE_NDF, 
        **fixed_paramsDQE, 
        Fnum=fn, gating=True, bn=True, method=DQNDF.CNNRNN, 
        evaluate=e,
    )

#     bestDQANDFs.append(bestDQANDF)
    bestDQSNDFs.append(bestDQSNDF)
    bestDQENDFs.append(bestDQENDF)
    print('---')

In [ ]:
# pickle.dump(bestDQANDFs, open('memoryDQANDFs.p', 'wb'))
# pickle.dump(bestDQSNDFs, open('memoryDQSNDFs.p', 'wb'))
# pickle.dump(bestDQENDFs, open('memoryDQENDFs.p', 'wb'))

## Save Final Result

In [ ]:
memoryNDs = pickle.load(open('memoryNDs.p', 'rb'))
print('ND len', len(memoryNDs))

bestDQAs = pickle.load(open('memoryDQANDFs.p', 'rb'))
bestDQSs = pickle.load(open('memoryDQSNDFs.p', 'rb'))
bestDQEs = pickle.load(open('memoryDQENDFs.p', 'rb'))
print('DQ len', len(bestDQAs))

bestDQAsNDF = pickle.load(open('memoryDQANDFs.p', 'rb'))
bestDQSsNDF = pickle.load(open('memoryDQSNDFs.p', 'rb'))
bestDQEsNDF = pickle.load(open('memoryDQENDFs.p', 'rb'))
print('DQNDF len', len(bestDQAsNDF))

In [ ]:
# submission[0] = ND[1], DQA[1], DQSNDF[3], DQENDF[3]

In [ ]:
testday = '190103'
i = 0
datahelper.pred_to_submission(
    memoryNDs[1], 
    bestDQAs[1], bestDQSsNDF[3], bestDQEsNDF[3], 
    test_turns, testIDs, 
    filename='Testing/{}upload/{}.json'.format(testday, i),
)

In [ ]:
testday = 'TESTDAY'
BEST_PATH = 'best_{}/'.format(testday)
bestND = pickle.load(open(BEST_PATH + 'bestND.p', "rb"))
bestDQAs = pickle.load(open(BEST_PATH + 'bestDQAs.p', "rb"))
bestDQSs = pickle.load(open(BEST_PATH + 'bestDQSs.p', "rb"))
bestDQEs = pickle.load(open(BEST_PATH + 'bestDQEs.p', "rb"))

In [ ]:
pred_to_submission(bestNDs[0], bestDQAs[0], bestDQSs[0], bestDQEs[0], test_turns, testIDs, filename='nd3.json')

In [ ]:
a = tf.constant(np.array([[1, 2], [3, 4], [5, 6]]), dtype=tf.float32)
b = tf.constant(np.array([[1, 2], [3, 4], [5, 6]]), dtype=tf.float32)
# c = tf.tensordot( a, b , axes=1)
c = tf.reduce_sum( tf.multiply( a, b ), axis=1)
# d = tf.scalar_mul([1,2], a)

x = tf.constant([[5.0, 10.0, 15, 20], [5,10,15,20]])
y = tf.multiply(x, [1,2,3,4])

# ? = 3
a_w = tf.constant(np.array([1,2,3,4,5]), dtype=tf.float32)
o_m = tf.constant(np.array([[1,2,3,4,5,6,7], 
                            [1,2,3,4,5,6,7],
                            [1,2,3,4,5,6,7],
                            [1,2,3,4,5,6,7],
                            [1,2,3,4,5,6,7]]), 
                            dtype=tf.float32,
                           )
b = tf.constant([2,3])
c = tf.reshape(b, [-1,1,1])
# out = tf.scalar_mul(a_w, o_m)
# out = tf.reduce_sum(tf.multiply(a_w, o_m), axis=0)

stack = [[1,2,3], [4,5,6], [7,8,9]]

with tf.Session() as sess:
#     output = sess.run(out)
    print(sess.run(c))
#     output = sess.run(d)
#     print(output)
#     print(sess.run(y))
#     print(sess.run(tf.stack(stack)))

In [ ]:
a = tf.constant([[1,1,1], [2,2,2], [3,3,3], [4,4,4], [5,5,5], [6,6,6], [7,7,7]])  # 7x3 (7x1024)
b = tf.reduce_mean(a, axis=0)
# b = tf.constant([1,2,3,4,5,6,7]) # (7, )
# c = tf.reshape(b, [-1,1])
# d = tf.multiply(a,c)
with tf.Session() as sess:
#     print(sess.run(a).shape)
    print(sess.run(b))
#     print(sess.run(d).shape)
#     print(sess.run(d))